In [2]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from itables import show


## Import pre-cleaned data and find most viable "EARN" variables

In [3]:
df = pd.read_csv('Data/clean_field_of_study.csv', dtype={'CIPFIELD': str, 'CIPCODE': str})

# Look at which "EARN" variables have the most datapoints
earn_col_counts = {}
for col in df.columns:
    if "EARN" in col:
        earn_col_counts[col] = len(df[col].unique()) -1 # subtract 1 to account for NaN

# sort dictionary by value to see which variables have the most data
earn_col_counts = {k: v for k, v in sorted(earn_col_counts.items(), key=lambda item: item[1], reverse=True)}
list(earn_col_counts.items())[:5] # Display first 5 items

[('EARN_MDN_HI_1YR', 264),
 ('EARN_MDN_1YR', 254),
 ('EARN_MDN_4YR', 244),
 ('EARN_PELL_WNE_MDN_1YR', 226),
 ('EARN_NE_MDN_3YR', 219)]

## Filter data down to the two key variables discovered above
* EARN_MDN_1YR: 'Median earnings of graduates working and not enrolled 1 year after completing'   
* EARN_MDN_4YR: 'Median earnings of graduates working and not enrolled 4 years after completing'

In [4]:
# We decide to us EARN_MDN_1YR and EARN_MDN_4YR
df = df[df.columns.to_list()[:7] + ['EARN_MDN_1YR', 'EARN_MDN_4YR']] 
df.to_csv('Data/ouchie.csv', index=False)

# Get counts of nans
both_nan_count = df[(df['EARN_MDN_1YR'].isna()) & (df['EARN_MDN_4YR'].isna())].shape[0]
either_nan_count = df[(df['EARN_MDN_1YR'].isna()) | (df['EARN_MDN_4YR'].isna())].shape[0] - both_nan_count
print(f'Total Row Count: {len(df)}\nBoth NaN: {both_nan_count}\nEither NaN: {either_nan_count}')

# Drop rows where both EARN vars are nan (for easier viewing)
df.dropna(inplace=True)
# df = df.dropna(subset=['EARN_MDN_1YR', 'EARN_MDN_4YR'], how='all')
show(df)

Total Row Count: 554
Both NaN: 262
Either NaN: 85


## Look at average earnings by University, by Field, and by Field taken at a given University

In [5]:
# View average median earnings for entire universities (mean of all programs with public record)
univ_group_mean = df.groupby('INSTNM').agg({'EARN_MDN_1YR': 'mean', 'EARN_MDN_4YR': 'mean'}).reset_index()
univ_group_mean = univ_group_mean.sort_values(by='EARN_MDN_4YR', ascending=False)
univ_group_mean

,INSTNM,EARN_MDN_1YR,EARN_MDN_4YR
0,Brigham Young University,"47,174.15","63,373.25"
2,University of Utah,"47,227.86","61,987.43"
6,Westminster College,"46,325.40","60,817.40"
4,Utah Valley University,"44,682.09","57,600.16"
5,Weber State University,"46,260.09","55,969.97"
3,Utah State University,"39,905.87","52,193.71"
1,Southern Utah University,"35,300.25","44,503.17"


In [6]:
# View average median earnings for each field of study (mean of all programs @all universities with public record)
field_group_mean = df.groupby(['CIPDEF']).agg({'EARN_MDN_1YR': 'mean', 'EARN_MDN_4YR': 'mean'}).reset_index()
field_group_mean = field_group_mean.sort_values(by='EARN_MDN_4YR', ascending=False)
field_group_mean

,CIPDEF,EARN_MDN_1YR,EARN_MDN_4YR
7,COMPUTER AND INFORMATION SCIENCES AND SUPPORT ...,"69,317.67","88,258.00"
10,ENGINEERING.,"69,412.13","87,750.60"
19,MATHEMATICS AND STATISTICS.,"57,342.50","79,298.00"
8,CONSTRUCTION TRADES.,"68,886.00","78,807.00"
11,ENGINEERING/ENGINEERING-RELATED TECHNOLOGIES/T...,"65,609.00","78,380.50"
6,COMMUNICATIONS TECHNOLOGIES/TECHNICIANS AND SU...,"48,703.00","74,650.00"
4,"BUSINESS, MANAGEMENT, MARKETING, AND RELATED S...","55,107.69","72,721.94"
26,TRANSPORTATION AND MATERIALS MOVING.,"50,560.50","70,107.50"
22,PHYSICAL SCIENCES.,"50,117.33","69,189.33"
1,ARCHITECTURE AND RELATED SERVICES.,"44,067.50","62,267.00"


In [9]:
# View average median earnings for each field of study for each university
univ_group = df.groupby(['INSTNM', 'CIPDEF']).agg({'EARN_MDN_1YR': 'mean', 'EARN_MDN_4YR': 'mean'}).reset_index()
show(univ_group)
pivot_1yr = univ_group.pivot(index='INSTNM', columns='CIPDEF', values='EARN_MDN_1YR')
pivot_4yr = univ_group.pivot(index='INSTNM', columns='CIPDEF', values='EARN_MDN_4YR')
show(pivot_1yr, caption="1 Year Median Earnings by Field of Study")
show(pivot_4yr, caption="4 Year Median Earnings by Field of Study")